In [10]:
import pandas as pd
import requests
from fonctions import *

In [11]:
# Création BDD
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import *
from fonctions import *

engine = create_engine('sqlite:///db.sqlite')
Base = declarative_base()

class Objet(Base):
    __tablename__ = "Objet"

    recordid = Column(String, primary_key=True)
    gare_code_uic = Column(String, nullable=True)
    type = Column(String, nullable=True)
    nature = Column(String, nullable=True)
    recordtype = Column(String, nullable=True)
    date = Column(String, nullable=True)

class Gare(Base):
    __tablename__ = "Gare"
    
    uic_code = Column(String, primary_key=True)
    departement_libellemin = Column(String, nullable=True)
    gare_regionsncf_libelle = Column(String, nullable=True)
    gare_alias_libelle_noncontraint = Column(String, nullable=True)
    adresse_cp = Column(String, nullable=True)
    code_gare = Column(String, nullable=True)
    lat = Column(REAL, nullable=True)
    long = Column(REAL, nullable=True)

class Meteo(Base):
    __tablename__ = "Meteo"
    
    id = Column(String, primary_key=True)
    date = Column(String, primary_key=True)
    nom_reg = Column(String)
    nom_dept = Column(String)
    lat = Column(REAL)
    long = Column(REAL)
    tc = Column(REAL)
    code_dep = Column(String)
    mois_de_l_annee = Column(String)

class Frequence(Base):
    __tablename__ = "Frequence"

    recordid = Column(String, nullable=True, primary_key=True)
    total_voyageurs_2016 = Column(Integer)
    total_voyageurs_2017 = Column(Integer)
    total_voyageurs_2018 = Column(Integer)
    total_voyageurs_2019 = Column(Integer)
    total_voyageurs_2020 = Column(Integer)
    total_voyageurs_2021 = Column(Integer)
    code_uic_complet = Column(String)
    nom_gare  = Column(String)
    region = Column(String)
    region_name = Column(String)
    
Base.metadata.create_all(engine)

In [12]:
Session = sessionmaker(bind=engine)
session = Session()

In [13]:
years = [2016, 2017, 2018, 2019, 2020, 2021]
url = "https://ressources.data.sncf.com/api/records/1.0/search/"

In [14]:
# # Récupération des données d'objets trouvés
# for a in years:
#     for m in ["01","02","03","04","05","06","07","08","09","10","11","12"]:
#         print(f'\n PERTES Status : {m}-{str(a)} en cours...\n ')
#         response = requests.get(f'{url}?dataset=objets-trouves-restitution&rows=-1&refine.date={str(a)}%2F{m}').json()
#         try:
#             for i in response['records']:
#                 try:
#                     gare_code_uic = i['fields']['gc_obo_gare_origine_r_code_uic_c']
#                 except:
#                     gare_code_uic = None
                    
#                 session.add(Objet(recordid = i['recordid'],gare_code_uic = gare_code_uic,type = i['fields']['gc_obo_type_c'],nature = i['fields']['gc_obo_nature_c'],recordtype = i['fields']['gc_obo_nom_recordtype_sc_c'],date = i['fields']['date']))
#             session.commit()
#         except :
#             pass
# session.close()
# print(f'\n PERTES Status : terminé !\n ')

In [15]:
# # Récupération de toutes les gares
# print(f'\n GARES Status : en cours...\n ')
# for g in requests.get(f'{url}?dataset=referentiel-gares-voyageurs&q=&rows=-1&sort=gare_alias_libelle_noncontraint&facet=departement_libellemin&facet=segmentdrg_libelle&facet=gare_agencegc_libelle&facet=gare_regionsncf_libelle&facet=gare_ug_libelle').json()['records']:
#     try:
#         gare_regionsncf_libelle = g['fields']['gare_regionsncf_libelle']
#     except KeyError:
#         gare_regionsncf_libelle = None
#     try:
#         departement_libellemin = g['fields']['departement_libellemin']
#     except KeyError:
#         departement_libellemin = None
#     try:
#         gare_alias_libelle_noncontraint = g['fields']['gare_alias_libelle_noncontraint']
#     except KeyError:
#         gare_alias_libelle_noncontraint = None
#     try:
#         adresse_cp = g['fields']['adresse_cp']
#     except KeyError:
#         adresse_cp = None
#     try:
#         code_gare = g['fields']['code_gare']
#     except KeyError:
#         code_gare = None
#     try:
#         lat = g['geometry']['coordinates'][1]
#         long = g['geometry']['coordinates'][0]
#     except KeyError:
#         lat = None
#         long = None
#     try:
#         code_gare = g['fields']['code_gare']
#     except KeyError:
#         code_gare = None
#     try:
#         session.add(Gare(
#             uic_code = g['fields']['uic_code'],
#             departement_libellemin = departement_libellemin,
#             gare_regionsncf_libelle = gare_regionsncf_libelle,
#             gare_alias_libelle_noncontraint = gare_alias_libelle_noncontraint,
#             adresse_cp = adresse_cp,
#             code_gare = code_gare,
#             lat = lat,
#             long = long))
#         session.commit()   
#     except:
#         pass
# session.close()
# print(f'\n GARES Status : terminé !\n ')

In [16]:
# Récupération des météos par région

df_gare = pd.read_sql_table('Gare', 'sqlite:///db.sqlite')
df_gare['gare_regionsncf_libelle'] = df_gare['adresse_cp'].apply(lambda x: search_region(str(x)[:2]))
list_region = list(df_gare['gare_regionsncf_libelle'].unique())

for region in list_region:
    for y in years:
        print(f'\n METEO Status : en cours Région : {region} ({str(y)})... \n ')
        for m in ["01","02","03","04","05","06","07","08","09","10","11","12"]:
            response = requests.get(f"https://public.opendatasoft.com/api/records/1.0/search/?dataset=donnees-synop-essentielles-omm&rows=-1&sort=date&facet=date&facet=nom&facet=temps_present&facet=libgeo&facet=nom_epci&facet=nom_dept&facet=nom_reg&refine.nom_reg={region}&refine.date={y}%2F{m}").json()
            
            for i in response['records']:
                try:
                    tc = i['fields']['tc']
                except KeyError:
                    tc = None
                session.add(Meteo(
                    id = i["recordid"],
                    date = i['fields']['date'],
                    nom_reg = i['fields']['nom_reg'],
                    nom_dept = i['fields']['nom_dept'],
                    lat = i['fields']['coordonnees'][0],
                    long = i['fields']['coordonnees'][1],
                    tc = tc,
                    code_dep = i['fields']['code_dep'],
                    mois_de_l_annee = i['fields']['mois_de_l_annee']
                ))
            session.commit()
session.close()
print(f'\n METEO Status : terminé ! \n ')


 METEO Status : en cours Région : Hauts-de-France (2016)... 
 

 METEO Status : en cours Région : Hauts-de-France (2017)... 
 

 METEO Status : en cours Région : Hauts-de-France (2018)... 
 

 METEO Status : en cours Région : Hauts-de-France (2019)... 
 

 METEO Status : en cours Région : Hauts-de-France (2020)... 
 

 METEO Status : en cours Région : Hauts-de-France (2021)... 
 

 METEO Status : en cours Région : Pays de la Loire (2016)... 
 

 METEO Status : en cours Région : Pays de la Loire (2017)... 
 

 METEO Status : en cours Région : Pays de la Loire (2018)... 
 

 METEO Status : en cours Région : Pays de la Loire (2019)... 
 

 METEO Status : en cours Région : Pays de la Loire (2020)... 
 

 METEO Status : en cours Région : Pays de la Loire (2021)... 
 

 METEO Status : en cours Région : Île-de-France (2016)... 
 

 METEO Status : en cours Région : Île-de-France (2017)... 
 

 METEO Status : en cours Région : Île-de-France (2018)... 
 

 METEO Status : en cours Région : Île-d

In [17]:
# # Récupération des Fréquences
# response = requests.get(f'{url}?dataset=frequentation-gares&q=&rows=-1&sort=nom_gare&facet=nom_gare&facet=code_postal&facet=segmentation_drg').json()
# print(f'\n FREQUENCE Status : en cours ... \n ')
# for i in response['records']:
#     try:
#         total_voyageurs_2016 = i['fields']['total_voyageurs_2016']
#     except:
#         total_voyageurs_2016 = None
#     try:
#         total_voyageurs_2017 = i['fields']['totalvoyageurs2017']
#     except:
#         total_voyageurs_2017 = None
        
#     try:
#         total_voyageurs_2018 = i['fields']['total_voyageurs_2018']
#     except:
#         total_voyageurs_2018 = None
        
#     try:
#         total_voyageurs_2019 = i['fields']['total_voyageurs_2019']
#     except:
#         total_voyageurs_2019 = None
        
#     try:
#         total_voyageurs_2020 = i['fields']['total_voyageurs_2020']
#     except:
#         total_voyageurs_2020 = None
        
#     try:
#         total_voyageurs_2021 = i['fields']['total_voyageurs_2021']
#     except:
#         total_voyageurs_2021 = None
#     try:
#         code_uic_complet = i['fields']['code_uic_complet']
#     except:
#         code_uic_complet = None
    
#     try:
#         nom_gare = i['fields']['nom_gare']
#     except:
#         nom_gare = None
        
    
#     session.add(
#         Frequence(
#             recordid = i['recordid'],
#             total_voyageurs_2016 = total_voyageurs_2016,
#             total_voyageurs_2017 = total_voyageurs_2017,
#             total_voyageurs_2018 = total_voyageurs_2018,
#             total_voyageurs_2019 = total_voyageurs_2019,
#             total_voyageurs_2020 = total_voyageurs_2020,
#             total_voyageurs_2021 = total_voyageurs_2021,
#             code_uic_complet = code_uic_complet,
#             nom_gare = nom_gare,
#             region = i['fields']['code_postal'][:2],
#             region_name = search_region(i['fields']['code_postal'][:2])
        
#         ))
#     session.commit()
# session.close()
# print(f'\n FREQUENCE Status : terminé ! \n ')


 FREQUENCE Status : en cours ... 
 

 FREQUENCE Status : terminé ! 
 


In [18]:
# Meteo.__table__.drop(engine)

In [19]:
# Frequence.__table__.drop(engine)

NameError: name 'μ' is not defined